<a href="https://colab.research.google.com/github/SarahGoddaer/Machine_Learning_course_UGent_D012554_kaggle/blob/master/5%3A%20Random_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn import metrics

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Decision trees

Next I will try to fit a decision tree model to do predictions on the testset. An advantage about this kind of modelling is that this doesn't need scaling the data.

In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")
testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

features = trainset.copy()
features.pop('label')
feature_names = list(features.columns)

test_features = testset.copy()
test_features.pop('index')
test_feature_names = list(test_features.columns)

In [0]:
from sklearn.tree import DecisionTreeClassifier
model =  DecisionTreeClassifier(criterion='entropy')

In [5]:
score_train = cross_val_score(model, features, trainset.label,scoring='roc_auc', cv= 10).mean()
print('AUC score for trainset: '+ str(score_train))

AUC score for trainset: 0.7532918712276511


From what I've seen in the SVM model is this not a score that I'm very happy with.. But who knows, maybe I can optimize it a bit more...

In [7]:
from sklearn.model_selection import GridSearchCV
search_space = {'max_depth':[4,6,7,8,9,10], 'splitter':['best','random']}

grid_search = GridSearchCV(model, param_grid=search_space, scoring='roc_auc', cv=10)
grid_search.fit(features, trainset.label)

print(grid_search.best_params_)

means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']

for mean_score, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print("{:.3f} (+/-{:.3f}) for {}".format(mean_score, std * 2, params))

{'max_depth': 8, 'splitter': 'best'}
0.736 (+/-0.056) for {'max_depth': 4, 'splitter': 'best'}
0.670 (+/-0.086) for {'max_depth': 4, 'splitter': 'random'}
0.778 (+/-0.051) for {'max_depth': 6, 'splitter': 'best'}
0.727 (+/-0.084) for {'max_depth': 6, 'splitter': 'random'}
0.785 (+/-0.044) for {'max_depth': 7, 'splitter': 'best'}
0.761 (+/-0.060) for {'max_depth': 7, 'splitter': 'random'}
0.796 (+/-0.050) for {'max_depth': 8, 'splitter': 'best'}
0.756 (+/-0.074) for {'max_depth': 8, 'splitter': 'random'}
0.793 (+/-0.062) for {'max_depth': 9, 'splitter': 'best'}
0.770 (+/-0.069) for {'max_depth': 9, 'splitter': 'random'}
0.783 (+/-0.074) for {'max_depth': 10, 'splitter': 'best'}
0.789 (+/-0.040) for {'max_depth': 10, 'splitter': 'random'}


In [8]:
model = DecisionTreeClassifier(max_depth=8, criterion = 'entropy') 
score_train = cross_val_score(model, features, trainset.label,scoring='roc_auc', cv= 10).mean()
print('AUC score for trainset: '+ str(score_train))

AUC score for trainset: 0.7987575340052404


This is an increase of 4.5%, so I think this is not bad, but the actual AUC score doesn't even come close to what I found with the SVM model. I'm going again to a more complicated model:

#Random forests

In [9]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='entropy')

score_train = cross_val_score(rf, features, trainset.label,scoring='roc_auc', cv= 10).mean()
print('AUC score for trainset: '+ str(score_train))

AUC score for trainset: 0.9281905153969374


Waaaw, this is a high score immediatly without optimizing it's parameters. This is a model that is known for it's good performance, without optimizing it's parameters. I'm going to try to lift the score, maybe it will be higher than the SVM model, we'll see...

First I want to know which parameters are included:

In [13]:
print(rf.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}


In [11]:
from sklearn.model_selection import GridSearchCV

search_space = {'max_depth': [5,10,15, 20, 30, 50],
                'n_estimators': [200, 250, 300, 400]}

grid_search = GridSearchCV(rf,search_space)
grid_search.fit(features,trainset.label)

print(grid_search.best_estimator_)
print(grid_search.best_params_)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=50, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
{'max_depth': 50, 'n_estimators': 300}


In [16]:
model = RandomForestClassifier(criterion='entropy',n_estimators=250, max_depth=20, bootstrap=False)

score_train = cross_val_score(model, features, trainset.label,scoring='roc_auc', cv= 10).mean()
print('AUC score for trainset: '+ str(score_train))

AUC score for trainset: 0.9375779821605509


OK, i was able to higher the score a little bit, but not so much as I expected. Maybe I should do it's tuning better or maybe it's already so good without changing its parameters that this doesn't matter so much.